In [1]:
import pandas as pd
import geopandas as gpd
from access import Access, weights, Datasets

In [2]:
# Datasets.available_datasets()

In [3]:
chicago_pop = Datasets.load_data("chi_pop")

In [4]:
chi_doc = Datasets.load_data("chi_doc_geom")

In [5]:
chi_times = Datasets.load_data('chi_times')

In [6]:
A = Access(demand_df            = chicago_pop,
           demand_index         = 'geoid',
           demand_value         = 'pop',
           supply_df            = chi_doc,
           supply_index         = 'geoid',
           supply_value         = 'doc',
           cost_df              = chi_times,
           cost_origin          = 'origin',
           cost_dest            = 'dest',
           cost_name            = 'cost',
           neighbor_cost_df     = chi_times,
           neighbor_cost_origin = 'origin',
           neighbor_cost_dest   = 'dest',
           neighbor_cost_name   = 'cost')

In [7]:
fn30 = weights.step_fn({10 : 1, 20 : 0.68, 30 : 0.22})
fn60 = weights.step_fn({20 : 1, 40 : 0.68, 60 : 0.22})
gaussian = weights.gaussian(20)
gravity = weights.gravity(scale = 60, alpha = -1)

In [8]:
A.weighted_catchment    (name = "gravity",  weight_fn = gravity)
A.fca_ratio             (name = "fca",      max_cost = 15)
A.fca_ratio             (name = "fca",      max_cost = 30) # Note - the warning -- good!
A.fca_ratio             (name = "fca60",    max_cost = 60)
A.fca_ratio             (name = "fca90",    max_cost = 90)
A.two_stage_fca         (name = "2sfca",    max_cost = 60)
A.enhanced_two_stage_fca(name = "2sfca30",  weight_fn = fn30)
A.enhanced_two_stage_fca(name = "2sfca60",  weight_fn = fn60)
A.enhanced_two_stage_fca(name = "g2sfca",   weight_fn = gaussian)
A.three_stage_fca       (name = "3sfca")
A.raam(name = "raam", tau = 60)
A.raam(name = "raam30", tau = 30)

/data/cigi/cybergisx-easybuild/conda/python3-0.9.0/lib/python3.8/site-packages/access/fca.py:137: UserWarning: some tracts may be unaccounted for in supply_cost
  warnings.warn("some tracts may be unaccounted for in supply_cost", stacklevel = 1)
access INFO     :: Overwriting fca_doc.


,raam30_doc
geoid,
17031010100,1.027597
17031010201,0.940239
17031010202,1.031144
17031010300,1.071731
17031010400,1.317185
...,...
17197883903,2.248139
17197883904,2.026473
17197884101,2.365171


In [9]:
A.access_df.head()

,pop,gravity_doc,fca_doc,fca60_doc,fca90_doc,2sfca_doc,2sfca30_doc,2sfca60_doc,g2sfca_doc,3sfca_doc,raam_doc,raam30_doc
geoid,,,,,,,,,,,,
17031010100,4854,10618.895243,0.001640,0.001098,0.000922,0.000697,0.000980,0.000688,0.000765,0.001447,0.888102,1.027597
17031010201,6450,10801.211105,0.001536,0.001083,0.000910,0.000754,0.001094,0.000751,0.000820,0.001487,0.857214,0.940239
17031010202,2818,11324.424819,0.001521,0.001071,0.000914,0.000717,0.001030,0.000719,0.000800,0.001420,0.897102,1.031144
17031010300,6236,10535.149629,0.001573,0.001091,0.000916,0.000703,0.001031,0.000709,0.000791,0.001479,0.961906,1.071731
17031010400,5042,10934.394373,0.001551,0.001054,0.000911,0.000743,0.000910,0.000788,0.000876,0.001274,1.041687,1.317185


In [10]:
A.access_df.to_csv("access_result.csv")